In [3]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [4]:
pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.0 MB/s eta 0:00:00


In [6]:
pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.2 MB/s eta 0:00:00


In [10]:
pip install PyPdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 3.7 MB/s eta 0:00:00


In [ ]:
pip install langchain-pinecone

In [ ]:
pip install langchain pinecone-client cohere

In [11]:
#Importing Packages
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.embeddings import CohereEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import Cohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_pinecone import PineconeVectorStore

In [12]:
#Loading the PDF
loader=PyPDFLoader('content.pdf')
doc=loader.load()

In [13]:
#Splitting the Content into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(doc)

In [57]:
#Creating Pincone Index
pc = Pinecone(api_key="ENTER_KEY")
index_name = "quickstart"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [54]:
#Creating Embeddings using Cohere Embeddings
load_dotenv()
os.environ['COHERE_API_KEY'] =  'ENTER_KEY'
os.environ['PINECONE_API_KEY'] =  'ENTER_KEY'
embeddings = CohereEmbeddings(model = "embed-english-light-v3.0",user_agent="my-app/1.0")
vector_store=PineconeVectorStore(index_name=index_name,embedding=embeddings)

In [58]:
#Upserting 50 documents into Pinecone Index
vector_store.from_documents(chunks[:50],index_name=index_name,embedding=embeddings)

In [60]:
#Invoking Cohere Model
llm_cohere = Cohere(model = "command-r-plus-08-2024", temperature=0.9)

In [61]:
prompt = ChatPromptTemplate.from_template(
    """Answer the following questions based on the provided context.
    Think step by step before providing a detailed answer.
    <context>{context}</context>
    Question: {input}"""
)
doc_chain_cohere=create_stuff_documents_chain(llm_cohere,prompt)

In [62]:
retriver=vector_store.as_retriever()
rc=create_retrieval_chain(retriver,doc_chain_cohere)

In [65]:
response=rc.invoke({"input":"Summarise the document"})
response['answer']

"Sure! Here is a summary of the document:\n\nThe document outlines the second part of a two-part project for developing a Question-Answering (QA) system. In Part 1, the focus was on building a backend system and creating a Colab notebook to demonstrate the entire pipeline, from data loading to question answering. Part 2 is about creating an interactive interface for this QA system.\n\nKey Deliverables and Tasks:\n- Develop a simple frontend interface using Streamlit or Gradio, enabling users to upload PDF documents and ask questions.\n- Integrate the backend developed in Part 1 to handle PDF processing, store document embeddings, and provide real-time answers to user queries.\n- Ensure the system's efficiency and accuracy in handling multiple queries, with contextually relevant responses.\n- Allow users to see the relevant document segments that the system uses to generate answers.\n- Containerize the application using Docker for easy deployment.\n- Provide a deployed QA bot with this 